In [1]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import solve_ivp 
from matplotlib import pyplot as plt

The distance from $L_i$, $i = 1, 2$ to the smaller primary is given by the unique positive solution $\gamma_i$ of the following equation:
$$\gamma^5 \mp (3-\mu)\gamma^4 + (3-2\mu)\gamma^3 -\mu\gamma^2 \pm2\mu\gamma-\mu=0,$$
where the upper sign is for $\gamma_1$ and the lower one for $\gamma_2$.

$$\mu = \frac{M_s}{M_p + M_s},$$
where $M_p$ and $M_s$ denote the masses of the primary and the secondary (smaller of the two primaries) body, respectively.

The equations of motion in the barycentric reference frame (nondimentionlized):
\begin{align}
\ddot{x} - 2\dot{y} - x &= - \frac{(1-\mu)(x+\mu)}{r_p^3} - \frac{\mu(x-(1-\mu))}{r_s^3}, \\
\ddot{y} + 2\dot{x} - y &= - \frac{(1-\mu)y}{r_p^3} - \frac{\mu y}{r_s^3}, \\
\ddot{z} &= - \frac{(1-\mu)z}{r_p^3} - \frac{\mu z}{r_s^3},
\end{align}
where
$$r_p = \sqrt{(x+\mu)^2 + y^2 + z^2}, \quad r_s = \sqrt{(x-(1-\mu))^2 + y^2 + z^2}$$

The conversion from units of distance, velocity, and time in the unprimed, normalized system to the primed, dimensionalized system is:
\begin{align}
d' &= L\cdot d, \\
s' &= V\cdot s, \\
t' &= \frac{T}{2\pi}t,
\end{align}
where where $L$ is the distance between the centers of $M_P$ and $M_S$, $V$ is the orbital velocity of $M_S$, $T$ is the orbital period of $M_P$ and $M_S$.

In [2]:
G = 6.67408e-20
# https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de430_and_de431.pdf

class three_BP:
    def __init__(self, type='EM'):
        if type=='EM': #Earth-Moon
            self.Mp = 398600.4354 / G  #Earth   index "p" stands for the primary body
            self.Ms = 4902.80007 / G   #Moon    index "s" stands for the secondary body
            self.a = 384399.014  # [km] mean SMA
        elif type=='SE': #Sun-Earth
            self.Mp = 132712440041.939400 / G  #Sun
            self.Ms = 398600.435436 / G  #Earth
            self.a = 149597870.700 # [km] mean SMA           
        else:
            raise Exception('Unknown 3BP configuration')
            
        self.mu = self.Ms / (self.Ms + self.Mp)
        self.Rs = self.a * (1. - self.mu) 
        self.Rp = self.a * self.mu
        self.mean_motion = np.sqrt(G * (self.Ms + self.Mp) / (self.a)**3)

    def get_L1(self):

        return (1. - self.mu - self.get_L1_quintic()) * self.a
    
    def get_L2(self):

        return (1. - self.mu + self.get_L2_quintic()) * self.a
    
    def get_L1_quintic(self):
        def quintic_L1(x):
            return x**5 - (3. - self.mu) * x**4 + (3. - 2. * self.mu) * x**3 - self.mu * x**2 + 2. * self.mu * x - self.mu
        
        root = fsolve(quintic_L1, (1. - self.mu) / 2., xtol=1e-10)
        return root[0]  

    def get_L2_quintic(self):
        def quintic_L2(x):
            return x**5 + (3. - self.mu) * x**4 + (3. - 2. * self.mu) * x**3 - self.mu * x**2 - 2. * self.mu * x - self.mu
        
        root = fsolve(quintic_L2, (1. - self.mu), xtol=1e-10)
        return root[0]      
    
    def tbp_rhs(self, t, x):
        # dimensioness barycentric coordinates
        dxdt = np.zeros(6)
        
        rP3 = np.sqrt((x[0] + self.mu)**2 + x[1]**2 + x[2]**2)
        rS3 = np.sqrt((x[0] - 1. + self.mu)**2 + x[1]**2 + x[2]**2)
        
        dxdt[:3] = x[3:]
        dxdt[3] = 2. * x[4] + x[0] - (1 - self.mu) * (x[0] + self.mu) / rP3**3 - self.mu * (x[0] - 1 + self.mu) / rS3**3 
        dxdt[4] = - 2. * x[3] + x[1] - (1 - self.mu) * x[1] / rP3**3 - self.mu * x[1] / rS3**3
        dxdt[5] = - (1 - self.mu) * x[2] / rP3**3 - self.mu * x[2] / rS3**3
        
        return dxdt

def richardson_traj(tau_1, cnst, delta_n=1):
    X = np.zeros((np.size(tau_1), 3))
    
    X[:, 0] = cnst.a21 * cnst.Ax**2 + cnst.a22 * cnst.Az**2 - cnst.Ax * np.cos(tau_1) +\
        (cnst.a23 * cnst.Ax**2 - cnst.a24 * cnst.Az**2) * np.cos(2. * tau_1) +\
        (cnst.a31 * cnst.Ax**3 - cnst.a32 * cnst.Ax * cnst.Az**2) * np.cos(3. * tau_1)
    X[:, 1] = cnst.k * cnst.Ax * (np.sin(tau_1)) +\
        (cnst.b21 * cnst.Ax**2 - cnst.b22 * cnst.Az**2) * (np.sin(2. * tau_1)) +\
        (cnst.b31 * cnst.Ax**3 - cnst.b32 * cnst.Ax * cnst.Az**2) * np.sin(3. * tau_1) 
    X[:, 2] = delta_n * (cnst.Az * np.cos(tau_1) + cnst.d21 * cnst.Ax * cnst.Az * (np.cos(2. * tau_1) - 3.) +\
        (cnst.d32 * cnst.Az * cnst.Ax**2 - cnst.d31 * cnst.Az**3) * np.cos(3. * tau_1))   
    
    return X

In [3]:
tbp = three_BP('EM')
rL1 = tbp.get_L1()
rL2 = tbp.get_L2()
rt = tbp.get_L2_quintic()
l2 = 1.- tbp.mu + rt
rL1, rL2, rt, l2*tbp.a, np.sqrt(1 / tbp.a**3), tbp.mean_motion

(321709.35166269745,
 444243.08292925765,
 0.16783274461306147,
 444243.08292925765,
 4.195914234026077e-09,
 2.6653246337605276e-06)

The complete third-order solution for periodic motion about any of the collinear points is given by:

\begin{array}{} 
x &= a_{21} A_x^2 + a_{22} A_z^2 - A_x \cos{\tau_1}+(a_{23}A_x^2 - a_{24}A_z^2)\cos{2\tau_1} + (a_{31}A_x^3-a_{32}A_x A_z^2)\cos{3\tau_1}, \\
y &= k A_x \sin{\tau_1} + (b_{21}A_x^2 - b_{22}A_z^2)\sin{2\tau_1} + (b_{31}A_x^3-b_{32}A_x A_z^2)\sin{3\tau_1}, \\
    z &= \delta_n A_z \cos{\tau_1}+\delta_n d_{21}A_x A_z(\cos{2\tau_1}-3) +\delta_n(d_{32}A_z A_x^2 - d_{31}A_z^3)\cos{3\tau_1},
\end{array}

where the switch function $\delta_n$ is given by
$$\delta_n = 2 - n, \quad n = 1,3,$$

$a_{ij}$, $b_{ij}$, $d_{ij}$ are constants (computed in the code below), and $\tau_1$ is defined as
$$\tau_1 = \lambda \tau + \phi, \quad \tau = \omega s, \quad s = n_1 t$$

Conversion from Richardson's coordinates to the barycentric:

In [4]:
class Parameters(object):
    pass

def halo_R3OA(Az, system="EM", point="L2"):
    def lambda_eqw(lmb):
        return lmb**4 + (c2 - 2.)*lmb**2 - (c2 - 1.) * (1. + 2. * c2)       
    
    # Get L points
    tbp = three_BP(system)
    rL = tbp.get_L2()
    
    n1 = np.sqrt(1 / tbp.a**3) # tbp.mean_motion    
    LU = rL - tbp.Rs  # length unit for L2
    gamma_L = LU / tbp.a
    
    if point == "L1":
        gamma_expr = gamma_L / (1. - gamma_L)
        c2 = (tbp.mu + (1. - tbp.mu) * gamma_expr**3) / gamma_L**3
        c3 = (tbp.mu - (1. - tbp.mu) * gamma_expr**4) / gamma_L**3
        c4 = (tbp.mu + (1. - tbp.mu) * gamma_expr**5) / gamma_L**3
    elif point == "L2":
        gamma_expr = gamma_L / (1. + gamma_L)
        c2 = (tbp.mu + (1. - tbp.mu) * gamma_expr**3) / gamma_L**3
        c3 = (-tbp.mu - (1. - tbp.mu) * gamma_expr**4) / gamma_L**3
        c4 = (tbp.mu + (1. - tbp.mu) * gamma_expr**5) / gamma_L**3
    
    root = fsolve(lambda_eqw, 1000)
    lmbda = root[(np.imag(root)==0 and root>0)][0]
        
    k = 2. * lmbda / (lmbda**2 + 1. - c2)
    
    d1 = 3. * lmbda**2 * (k * (6. * lmbda**2 - 1.) - 2. * lmbda) / k 
    d2 = 8. * lmbda**2 * (k * (11. * lmbda**2 - 1.) - 2. * lmbda) / k
    
    a21 = 3. * c3 * (k**2 - 2) / 4. / (1. + 2. * c2)
    a22 = 3. * c3 / 4. / (1. + 2 * c2)
    a23 = - 3. * c3 * lmbda * (3. * k**3 * lmbda - 6. * k * (k - lmbda) + 4.) / (4. * k * d1)
    a24 = - 3. * c3 * lmbda * (2. + 3. * k * lmbda) / (4. * k * d1)
        
    d21 = -c3 / 2. / lmbda**2
    d31 = 3. * (4. * c3 * a24 + c4) / 64. / lmbda**2
    d32 = 3. * (4. * c3 * (a23 - d21) + c4 * (4. + k**2)) / 64. / lmbda**2
    
    b21 = -3. * c3 * lmbda * (3. * k * lmbda - 4.) / 2. / d1
    b22 = 3. * c3 * lmbda / d1
    b31 = 3. * (8. * lmbda * (3. * c3 * (k*b21 - 2.*a23) - c4*(2. + 3.*k**2)) +\
                (9.*lmbda**2 + 1. + 2.*c2)*(4.*c3*(k*a23-b21)+k*c4*(4.+k**2))) / 8. / d2
    b32 = (9. * lmbda * (c3 * (k * b22 + d21 -2. * a24) - c4) +\
        3. * (9. * lmbda**2 + 1. + 2. * c2) * (4. * c3 * (k * a24 - b22 ) + k * c4) / 8.) / d2
    
    a31 = -9. * lmbda * (4. * c3 * (k * a23 - b21) + k * c4 * (4. + k**2)) / 4. / d2 +\
        (9. * lmbda**2 + 1. - c2) * (3. * c3 * (2. * a23 - k * b21) + c4 * (2. + 3. * k**2)) / 2. /d2
    a32 = -(9. * lmbda * (4. * c3 * (k * a24 - b22) + k * c4) / 4. +\
            1.5 * (9. * lmbda**2 + 1. - c2) * (c3 * (k * b22 + d21 - 2. * a24) - c4)) / d2
    
    denom = 2. * lmbda * (lmbda * (1. + k**2) - 2. *k)
    s1 = (1.5 * c3 * (2. * a21 * (k**2 - 2.) - a23 * (k**2 + 2.) - 2. * k * b21) -\
          3. * c4 * (3. * k**4 - 8* k**2 + 8.)/8.)/denom
    s2 = (1.5 * c3 * (2. * a22 * (k**2 - 2.) + a24 * (k**2 + 2.) + 2. * k * b22 + 5. * d21) +\
          3.* c4 * (12. - k**2) /8.)/denom

    a1 = -1.5 * c3 * (2. * a21 + a23 + 5. * d21) - 3. * c4 * (12. - k**2) / 8.
    a2 =  1.5 * c3 * (a24 - 2. * a22) + 9. * c4 / 8.
    
    l1 = a1 + 2. * lmbda**2 * s1
    l2 = a2 + 2. * lmbda**2 * s2
    
    Delta = lmbda**2 - c2
    
    Az /= LU 
    
    Ax = np.sqrt((-l2 * (Az)**2 - Delta) / l1)
    Ay = k * Ax
    
    omega = 1 + s1 * Ax**2 + s2 * (Az)**2
       
    theta = 0. 
    n_phase = 1. # can be 1 or 3
    delta_n = 2. - n_phase
    tau_1 = 0. + theta    # tau_z = 0. + (theta + np.pi * n / 2) this is taken care of by delta_n

    x0 = a21 * Ax**2 + a22 * Az**2 - Ax * np.cos(tau_1) +\
        (a23 * Ax**2 - a24 * Az**2) * np.cos(2. * tau_1) + (a31 * Ax**3 - a32 * Ax * Az**2) * np.cos(3. * tau_1)
    z0 = delta_n * (Az * np.cos(tau_1) + d21 * Ax * Az * (np.cos(2. * tau_1) - 3.) +\
        (d32 * Az * Ax**2 - d31 * Az**3) * np.cos(3. * tau_1))
    vy0 = k * Ax * (lmbda * omega * n1 * np.cos(tau_1)) +\
        (b21 * Ax**2 - b22 * Az**2) * (2. * lmbda * omega * n1 * np.cos(2. * tau_1)) +\
        (b31 * Ax**3 - b32 * Ax * Az**2) * 3. * lmbda * omega * n1 * np.cos(3. * tau_1)
       
    richardson = Parameters()
    richardson.gamma_L = gamma_L
    richardson.lmbda = lmbda
    richardson.k = k
    richardson.Delta = Delta
    richardson.c2 = c2
    richardson.c3 = c3
    richardson.c4 = c4
    richardson.s1 = s1
    richardson.s2 = s2
    richardson.l1 = l1
    richardson.l2 = l2
    richardson.a1 = a1
    richardson.a2 = a2
    richardson.d1 = d1
    richardson.d2 = d2
    richardson.a21 = a21
    richardson.a22 = a22
    richardson.a23 = a23
    richardson.a24 = a24
    richardson.a31 = a31
    richardson.a32 = a32
    richardson.b21 = b21
    richardson.b22 = b22
    richardson.b31 = b31
    richardson.b32 = b32
    richardson.d21 = d21
    richardson.d31 = d31
    richardson.d32 = d32
    richardson.Ax = Ax
    richardson.Az = Az
    
    return np.array([x0, 0, z0, 0, vy0, 0]), richardson

In [5]:
X0, cnst = halo_R3OA(2000, "EM")

if False:
    print(X0)
    print("gamma_L = ", cnst.gamma_L)
    print("lambda = ", cnst.lmbda)
    print("k = ", cnst.k)
    print("Delta = ", cnst.Delta)
    print("c2 = ", cnst.c2)
    print("c3 = ", cnst.c3)
    print("c4 = ", cnst.c4)
    print("s1 = ", cnst.s1)
    print("s2 = ", cnst.s2)
    print("l1 = ", cnst.l1)
    print("l2 = ", cnst.l2)
    print("a1 = ", cnst.a1)
    print("a2 = ", cnst.a2)
    print("d1 = ", cnst.d1)
    print("d2 = ", cnst.d2)
    print("a21 = ", cnst.a21)
    print("a22 = ", cnst.a22)
    print("a23 = ", cnst.a23)
    print("a24 = ", cnst.a24)
    print("a31 = ", cnst.a31)
    print("a32 = ", cnst.a32)
    print("b21 = ", cnst.b21)
    print("b22 = ", cnst.b22)
    print("b31 = ", cnst.b31)
    print("b32 = ", cnst.b32)
    print("d21 = ", cnst.d21)
    print("d31 = ", cnst.d31)
    print("d32 = ", cnst.d32)
    print("Ax = ", cnst.Ax)
    print("Az = ", cnst.Az)